In [4]:
from tensorflow import keras
from keras import layers, activations, Input
from keras.datasets import cifar10
import cv2
import numpy as np

In [5]:
def load_cifar10_data(img_rows, img_cols):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train[0:2000, :, :, :]
    y_train = y_train[0:2000]

    x_test = x_test[0:500, :, :, :]
    y_test = y_test[0:500]

    x_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_train[:, :, :, :]])
    x_test = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_test[:, :, :, :]])

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_cifar10_data(224, 224)
img_input = Input(shape=(224, 224, 3))

In [8]:
class ResidualUnit(layers.Layer):
    def __init__(self, filters, strides, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = activations.get(activation)
        self.main_layers = [layers.Conv2D(filters, (3,3),
            strides=strides, padding='same', use_bias=False),
            layers.BatchNormalization(),
            self.activation,
            layers.Conv2D(filters, (3,3), strides=1, padding='same', use_bias=False),
            layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1 :
            self.skip_layers = [
                layers.Conv2D(filters, (1,1), strides=strides, padding = 'same', use_bias=False),
                layers.BatchNormalization()]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [9]:
model = keras.models.Sequential()
model.add(layers.Conv2D(64, (7,7), strides=(2,2), input_shape=[224, 224, 3], padding='same', use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same'))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters ==prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(layers.GlobalAvgPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Epoch 1/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.1790 - loss: 3.3092 - val_accuracy: 0.1160 - val_loss: 88.9853
Epoch 2/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy: 0.3100 - loss: 1.8921 - val_accuracy: 0.1000 - val_loss: 6.3289
Epoch 3/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 205s 3s/step - accuracy: 0.3631 - loss: 1.7531 - val_accuracy: 0.2060 - val_loss: 2.8384


In [13]:
score = model.evaluate(x_test,y_test)
print("테스트 정확도 :", score[1])

16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 695ms/step - accuracy: 0.2022 - loss: 2.7710
테스트 정확도 : 0.20600000023841858
